In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [4]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [5]:
# add your code here - consider creating a new cell for each section of code
df = pd.merge(df_ratings, df_books, on='isbn')

# Filtrado más estricto para coincidir con el test
user_counts = df['user'].value_counts()
book_counts = df['isbn'].value_counts()

df = df[df['user'].isin(user_counts[user_counts >= 200].index)]
df = df[df['isbn'].isin(book_counts[book_counts >= 100].index)]

# Eliminar duplicados de usuario-libro (por si acaso)
df = df.drop_duplicates(['user', 'isbn'])

# Matriz libro-usuario
book_features = df.pivot_table(index='title', columns='user', values='rating').fillna(0)
mat_book_features = csr_matrix(book_features.values)

# Modelo KNN con parámetros ajustados
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
model_knn.fit(mat_book_features)

NearestNeighbors(algorithm='brute', metric='cosine')

In [12]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    recommended_books = [book, []]

    if book not in book_features.index:
        return recommended_books

    distances, indices = model_knn.kneighbors(
        book_features.loc[book].values.reshape(1, -1),
        n_neighbors=6
    )

    # Preparamos todas las recomendaciones
    recommendations = []
    for i in range(1, len(distances.flatten())):
        book_title = book_features.index[indices.flatten()[i]]
        distance = float(distances.flatten()[i])  # Convertimos a float estándar
        recommendations.append([book_title, distance])

    # Ordenamos por distancia (mayor primero)
    recommendations.sort(key=lambda x: x[1], reverse=True)

    # Nos aseguramos de devolver exactamente 5 recomendaciones
    recommended_books[1] = recommendations[:5]

    return recommended_books

In [13]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Tis: A Memoir', 0.7937096357345581], ['The Weight of Water', 0.7678344249725342], ['The Surgeon', 0.7669050693511963], ['I Know This Much Is True', 0.7646420001983643], ['The Lovely Bones: A Novel', 0.715856671333313]]]
You haven't passed yet. Keep trying!
